In [2]:
import open3d.ml as _ml3d
import open3d.ml.torch as ml3d

framework = 'torch'

In [9]:
cfg_file = "kitti.yml"
cfg = _ml3d.utils.Config.load_from_file(cfg_file)

In [12]:
import os
# download the weights.
ckpt_folder = "./logs/"
os.makedirs(ckpt_folder, exist_ok=True)
ckpt_path = ckpt_folder + "pointpillars_kitti_202012221652utc.pth"
pointpillar_url = "https://storage.googleapis.com/open3d-releases/model-zoo/pointpillars_kitti_202012221652utc.pth"


In [13]:
dataset = load_custom_dataset('trainval')

NameError: name 'load_custom_dataset' is not defined

In [ ]:
dataset = custom_dataset

model = ml3d.models.PointPillars(**cfg.model)

pipeline = ml3d.pipelines.ObjectDetection(model, dataset=dataset, device="gpu", **cfg.pipeline)





In [14]:
import open3d.ml as _ml3d
import open3d.ml.torch as ml3d
import os
import numpy as np
from glob import glob
from collections import defaultdict
from matplotlib import pyplot as plt
import open3d as o3d

In [15]:
files = glob('trainval/*/*_bbox.bin')

In [19]:
# point clouds and labels to pcd

labels = defaultdict(list)
for file in files:
    
    bbox = np.fromfile(file, dtype=np.float32)
    labels[bbox[9]].append(file)
    
    xyz = np.fromfile(file.replace('_bbox.bin', '_cloud.bin'), dtype=np.float32)
    xyz = xyz.reshape([3, -1])
    
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(xyz.T)
    
    o3d.io.write_point_cloud(file.replace('_bbox.bin', '.pcd'), pcd)

    

In [20]:
def load_custom_dataset():
    
    pcds = []
    
    for file in files:
        xyz = np.fromfile(file.replace('_bbox.bin', '_cloud.bin'), dtype=np.float32)
        xyz = xyz.reshape([3, -1])
        pcd = o3d.geometry.PointCloud()
        pcd.points = o3d.utility.Vector3dVector(xyz.T)
        pcds.append(pcd)
        
    return pcds
        

In [22]:
cfg_file = "kitti.yml"
cfg = _ml3d.utils.Config.load_from_file(cfg_file)


In [24]:
model = ml3d.models.PointPillars(**cfg.model, device='cpu')

In [25]:
custom_ds = load_custom_dataset()

In [27]:
pipeline = ml3d.pipelines.ObjectDetection(model, device="cpu", **cfg.pipeline)


In [28]:
ckpt_folder = "./logs/"
ckpt_path = ckpt_folder + "pointpillars_kitti_202012221652utc.pth"

pipeline.load_ckpt(ckpt_path=ckpt_path)

0

In [30]:
def prepare_point_cloud_for_inference(pcd):
    # Remove NaNs and infinity values
    pcd.remove_non_finite_points()
    # Extract the xyz points
    xyz = pcd.points

    # PointPillars classifier needs a 4th dimension (intensity), which my custom data does not have.
    # We add it here with default value of 0.5
    xyzi = []
    for point in xyz:
        xyzi.append(list(point) + [0.5])
    xyzi = np.array(xyzi)

    # Set the points to the correct format for inference
    data = {"point":xyzi, 'feat': None, 'label':np.zeros((len(xyz),), dtype=np.int32)}

    return data, pcd


data, pcd = prepare_point_cloud_for_inference(custom_ds[0])




In [34]:
import torch

pc = custom_ds[0]

data = {
    'point': pc,
    'full_point': pc,
    'feat': None,
    'calib': None,
    'bounding_boxes': None,
}


m = pipeline.model
with torch.no_grad():
    result = m(data)


AttributeError: 'dict' object has no attribute 'point'

In [45]:
base = ml3d.datasets.base_dataset.BaseDataset

AttributeError: module 'open3d.ml.torch.datasets' has no attribute 'base_dataset'

In [44]:
class MYDS(ml3d.datasets.Custom3D):
    def __init__(self, name="MYDS"):
        super().__init__(dataset, split=split)
        
    def get_split(self, split):
        return MyDatasetSplit(self, split=split)
    
    def is_tested(self, attr):
        # checks whether attr['name'] is already tested.

    def save_test_result(self, results, attr):
        # save results['predict_labels'] to file.
        return
    
    
class MyDatasetSplit():
    def __init__(self, dataset, split='train'):
        self.split = split
        self.path_list = []
        # collect list of files relevant to split.

    def __len__(self):
        return len(self.path_list)

    def get_data(self, idx):
        path = self.path_list[idx]
        points, features, labels = read_pc(path)
        return {'point': points, 'feat': features, 'label': labels}

    def get_attr(self, idx):
        path = self.path_list[idx]
        name = path.split('/')[-1]
        return {'name': name, 'path': path, 'split': self.split}
        
        
    
    